In [ ]:
#     Generate Robust Feature set and FacePrint for each session from individual openface metric files
#
# Change the directories and patient naming conventions to match your setup.
#
# Author: Stephen Heisig
#
#
from __future__ import division
#from sqlalchemy import *
import os
import sys
import glob
import subprocess
#import platform
import pipes
import re
import traceback
from decimal import Decimal
import pandas as pd
import numpy as np
import datetime
import scipy as sp
from scipy import signal
import matplotlib as mp
import matplotlib.pyplot as plt
from datetime import timedelta
import math
import time, datetime
import seaborn as sns
import sklearn.cluster as cluster
import numpy as np
import scipy.signal
import scipy.io.wavfile
from scipy import stats
from scipy.signal import savgol_filter
from collections import Counter
import pingouin as pg


# Input parameters: these are typically set based on the directory structure and file naming of the input
# dataset. 
#N.B. If you change any of these fields the data will not readily join with other users.
Diag = 'TRD'
Session = 'Monday'
Participant = 'MAP_1122'
Speakers = ['participant','interviewer']


InputDataDir = '/Users/Heisig/Library/CloudStorage/OneDrive-TheMountSinaiHospital/TRD_Mayberg-Murrough_Share/Results/'+Participant+'/'
study = 'Test'

print('genRobustFaceStats Leap V0.0')

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 1000)


"""
    Action Unit definitions...
AU number	FACS name	Muscular basis
0	Neutral face	
1	Inner brow raiser	frontalis (pars medialis)
2	Outer brow raiser	frontalis (pars lateralis)
4	Brow lowerer	depressor glabellae, depressor supercilii, corrugator supercilii
5	Upper lid raiser	levator palpebrae superioris, superior tarsal muscle
6	Cheek raiser	orbicularis oculi (pars orbitalis)
7	Lid tightener	orbicularis oculi (pars palpebralis)
8	Lips toward each other	orbicularis oris
9	Nose wrinkler	levator labii superioris alaeque nasi
10	Upper lip raiser	levator labii superioris, caput infraorbitalis
11	Nasolabial deepener	zygomaticus minor
12	Lip corner puller	zygomaticus major
13	Sharp lip puller	levator anguli oris (also known as caninus)
14	Dimpler	buccinator
15	Lip corner depressor	depressor anguli oris (also known as triangularis)
16	Lower lip depressor	depressor labii inferioris
17	Chin raiser	mentalis
18	Lip pucker	incisivii labii superioris and incisivii labii inferioris
19	Tongue show	
20	Lip stretcher	risorius w/ platysma
21	Neck tightener	platysma
22	Lip funneler	orbicularis oris
23	Lip tightener	orbicularis oris
24	Lip pressor	orbicularis oris
25	Lips part	depressor labii inferioris, or relaxation of mentalis or orbicularis oris
26	Jaw drop	masseter; relaxed temporalis and internal pterygoid
27	Mouth stretch	pterygoids, digastric
28	Lip suck	orbicularis oris

"""

sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

np.set_printoptions(precision=16)  
np.set_printoptions(threshold=1000)


#N.B. This is a bit confusing but the columns called gaze_angle_x and gaze_angle_y
# aren't really angles but are radians
# pose_Rx, pose_Ry, pose_Rz This can be seen as pitch (Rx), yaw (Ry), and roll (Rz) of the head box
#These columns are radians
gazeCols = ['frame',
            'face_id',
            'confidence',
            'success',
            'gaze_angle_x',
            'gaze_angle_y',
            'pose_Rx',
            'pose_Ry',
            'pose_Rz',
            'AU01_r',
            'AU02_r',
            'AU04_r',
            'AU05_r',
            'AU06_r',
            'AU07_r',
            'AU09_r',
            'AU10_r',
            'AU12_r',
            'AU14_r',
            'AU15_r',
            'AU17_r',
            'AU20_r',
            'AU23_r',
            'AU25_r',
            'AU26_r',
            'AU45_r',
            'AU01_c',
            'AU02_c',
            'AU04_c',
            'AU05_c',
            'AU06_c',
            'AU07_c',
            'AU09_c',
            'AU10_c',
            'AU12_c',
            'AU14_c',
            'AU15_c',
            'AU17_c',
            'AU20_c',
            'AU23_c',
            'AU25_c',
            'AU26_c',
            'AU45_c']
               
newGazeCols = ['frame',
            'face_id',
            'confidence',
            'success',
            'gaze_Rx',
            'gaze_Ry',
            'pose_Rx',
            'pose_Ry',
            'pose_Rz',
            'AU01_int',
            'AU02_int',
            'AU04_int',
            'AU05_int',
            'AU06_int',
            'AU07_int',
            'AU09_int',
            'AU10_int',
            'AU12_int',
            'AU14_int',
            'AU15_int',
            'AU17_int',
            'AU20_int',
            'AU23_int',
            'AU25_int',
            'AU26_int',
            'AU45_int',
            'AU01_p',
            'AU02_p',
            'AU04_p',
            'AU05_p',
            'AU06_p',
            'AU07_p',
            'AU09_p',
            'AU10_p',
            'AU12_p',
            'AU14_p',
            'AU15_p',
            'AU17_p',
            'AU20_p',
            'AU23_p',
            'AU25_p',
            'AU26_p',
            'AU45_p'] 

#This procedure does a np.diff type function but if there are gaps
#as defined by 0 entries in the success series it will discard the 'BAD'
#entries (meaning the ones which involved an unsuccessful entry) The idea
#is that openface metric entries for frames where the face wasn't successfully tracked 
#will be excluded from the diff result.              
def gapDiff(metric_series,successes_series):
    diff_series = np.diff(metric_series)
    mask1 = successes_series.iloc[1:]
    mask2 = successes_series.iloc[0:-1]
    mask1.reset_index(inplace=True, drop=True)
    mask2.reset_index(inplace=True, drop=True)
    gapMask = mask1.multiply(mask2) 
    theNoGapDiff = diff_series[gapMask==1] 
    return [diff_series,gapMask]
    
def computeBasicFeatures(successful_metrics,metric_name):
    featureSuffixes = ['_Mean','_Var','_Kur','_Skew','_Median','_qtl05','_qtl50','_qtl95' ]
    absFeatures = ['Vel', 'Accel', 'Jerk']
    featureNames = [metric_name+s for s in featureSuffixes]
    if metric_name.split('_')[-1] in absFeatures:
         mean = np.mean(np.absolute(successful_metrics))
    else:
        mean = np.mean(successful_metrics)
    var = np.var(successful_metrics)
    median = np.median(successful_metrics)
    kur = sp.stats.kurtosis(successful_metrics)
    skew = sp.stats.skew(successful_metrics)
    qtl05 = np.quantile(successful_metrics, .05)
    qtl50 = np.quantile(successful_metrics, .50)
    qtl95 = np.quantile(successful_metrics, .95)      
    features = [ mean, var, kur, skew, median, qtl05, qtl50, qtl95 ]
    print('basicFeatureNames: \n',featureNames)
    print('basicFeatures: \n',features)
    return [features,featureNames]  
    
#Given a dataFrame column generate and return the list of robust features    
# Ex metrics = genMotionFeatures(gazeDF['gaze_angle_x'],'gaze_Rx')
def genRobustMotionFeatures(metric_series,success_series,metric_name):
    featureSet = []
    featureNameSet = []
    featureSuffixes = ['_Mean','_Var','_Kur','_Skew','_Median','_qtl05','_qtl50','_qtl95' ]
    featureNames = [metric_name+s for s in featureSuffixes]
    successful_metrics = metric_series[success_series==1]
    #print('successful_metrics: \n',successful_metrics)
    basicFeatures, basicFeatureNames = computeBasicFeatures(successful_metrics,metric_name)
    featureSet.extend(basicFeatures)
    featureNameSet.extend(basicFeatureNames)
    successful_timestamps = timestamps[success_series==1]
    gradient1 =  np.gradient(successful_metrics, successful_timestamps)/30.0
    gradient2 =  np.gradient(gradient1, successful_timestamps)/30.0
    gradient3 =  np.gradient(gradient2, successful_timestamps)/30.0
    #Savitsky-Golay Filter the raw data to remove jitter
    windowLen = 13
    polynomial = 2
    successful_metrics_sg_filtered = savgol_filter(successful_metrics , windowLen, polynomial)
    #SJH TODO Why is this here: /30.0  ????????
    gradient1_sg_filtered =  np.gradient(successful_metrics_sg_filtered, successful_timestamps)/30.0
    gradient2_sg_filtered =  np.gradient(gradient1_sg_filtered, successful_timestamps)/30.0
    gradient3_sg_filtered =  np.gradient(gradient2_sg_filtered, successful_timestamps)/30.0 
    
    [velocity,velMask] = gapDiff(metric_series,success_series)
    noGapVelocity = velocity[velMask==1] 
    #print('np.absolute(noGapVelocity): \n',np.absolute(noGapVelocity))
    velMetricName = metric_name+'_Vel'
    basicFeatures, basicFeatureNames = computeBasicFeatures(noGapVelocity,velMetricName)
    featureSet.extend(basicFeatures)
    featureNameSet.extend(basicFeatureNames)
    
    meanVel = np.mean(np.absolute(noGapVelocity))
    [accel,accelMask] = gapDiff(velocity,velMask)
    #print('accel: \n',accel)
    #print('accelMask: \n',accelMask)
    
    noGapAccel = accel[accelMask==1] 
    print('np.absolute(noGapAccel): \n',np.absolute(noGapAccel))
    meanAccel = np.mean(np.absolute(noGapAccel))
    accelMetricName = metric_name+'_Accel'
    basicFeatures, basicFeatureNames = computeBasicFeatures(noGapAccel,accelMetricName)
    featureSet.extend(basicFeatures)
    featureNameSet.extend(basicFeatureNames)
    
    [jerk,jerkMask] = gapDiff(accel,accelMask)
    #print('jerk: \n',jerk)
    #print('jerkMask: \n',jerkMask)
    
    noGapJerk = jerk[jerkMask==1] 
    print('np.absolute(noGapJerk): \n',np.absolute(noGapJerk))
    meanJerk = np.mean(np.absolute(noGapJerk))
    jerkMetricName = metric_name+'_Jerk'
    basicFeatures, basicFeatureNames = computeBasicFeatures(noGapJerk,jerkMetricName)
    featureSet.extend(basicFeatures)
    featureNameSet.extend(basicFeatureNames)
    
    print('successful_metrics.shape,noGapVelocity.shape,noGapJerk.shape,noGapAccel.shape: ',successful_metrics.shape,noGapVelocity.shape,noGapJerk.shape,noGapAccel.shape)
    print('type(successful_metrics): ',type(successful_metrics))
    print('type(noGapVelocity): ',type(noGapVelocity))
    #successful_metricsDF = pd.DataFrame(successful_metrics,columns=['metric'])
    noGapVelocityDF = pd.DataFrame(noGapVelocity,columns=['noGapVelocity'])
    noGapAccelDF = pd.DataFrame(noGapAccel,columns=['noGapAccel'])
    noGapJerkDF = pd.DataFrame(noGapJerk,columns=['noGapJerk'])
    gradient1DF = pd.DataFrame(gradient1,columns=['gradient1'])
    gradient2DF = pd.DataFrame(gradient2,columns=['gradient2'])
    gradient3DF = pd.DataFrame(gradient3,columns=['gradient3'])
    successful_metrics_sg_filteredDF = pd.DataFrame(successful_metrics_sg_filtered,columns=['successful_metrics_sg_filtered'])
    gradient1_sg_filteredDF = pd.DataFrame(gradient1_sg_filtered,columns=['gradient1_sg_filtered'])
    gradient2_sg_filteredDF = pd.DataFrame(gradient2_sg_filtered,columns=['gradient2_sg_filtered'])
    gradient3_sg_filteredDF = pd.DataFrame(gradient3_sg_filtered,columns=['gradient3_sg_filtered'])
    print('successful_metrics.shape,noGapVelocityDF.shape,noGapJerkDF.shape,noGapAccelDF.shape: ',successful_metrics.shape,noGapVelocity.shape,noGapJerk.shape,noGapAccel.shape)
    allNoGapDF = pd.concat([successful_metrics,successful_metrics_sg_filteredDF,noGapVelocityDF,noGapAccelDF,noGapJerkDF,gradient1DF,gradient2DF,gradient3DF,gradient1_sg_filteredDF,gradient2_sg_filteredDF,gradient3_sg_filteredDF], axis=1)
    allNoGapDF.columns = ['successful_metric','successful_metrics_sg_filtered','noGapVelocity','noGapAccel','noGapJerk','gradient1','gradient2','gradient3','gradient1_sg_filtered','gradient2_sg_filtered','gradient3_sg_filtered']
    #noGapFileName = noGapFileDir+Session+"_"+metric_name+"_noGapVelAccelJerk.csv"
    #allNoGapDF.to_csv(noGapFileName, index=False)

    return [featureSet,featureNameSet]    

#Generate correlation features    
def genFacePrint(gazeDF,features,featureNames):

    #Calculate Activity features 
    featIndices = []
    #Explicitly state and find the feature name indices   
    corrFeaturesList1 = ['gaze_Rx_Vel_Mean', 'gaze_Ry_Vel_Mean', 'pose_Rx_Vel_Mean', 'pose_Ry_Vel_Mean', 'pose_Rz_Vel_Mean', 
                        'AU01_int_Mean', 'AU02_int_Mean', 'AU04_int_Mean', 'AU05_int_Mean', 'AU06_int_Mean', 
                        'AU07_int_Mean', 'AU09_int_Mean', 'AU10_int_Mean', 'AU12_int_Mean', 'AU14_int_Mean', 'AU15_int_Mean', 
                        'AU17_int_Mean', 'AU20_int_Mean', 'AU23_int_Mean', 'AU25_int_Mean', 'AU26_int_Mean', 'AU45_int_Mean', 
                        'AU01_p_PerCent', 'AU02_p_PerCent', 'AU04_p_PerCent', 'AU05_p_PerCent', 'AU06_p_PerCent', 'AU07_p_PerCent', 'AU09_p_PerCent', 
                        'AU10_p_PerCent', 'AU12_p_PerCent', 'AU14_p_PerCent', 'AU15_p_PerCent', 'AU17_p_PerCent', 'AU20_p_PerCent', 'AU23_p_PerCent',
                        'AU25_p_PerCent', 'AU26_p_PerCent', 'AU45_p_PerCent']
    corrFeaturesList2 = ['gaze_Rx_Vel_Mean', 'gaze_Ry_Vel_Mean', 'pose_Rx_Vel_Mean', 'pose_Ry_Vel_Mean', 'pose_Rz_Vel_Mean', 
                        'AU01_int_qtl90', 'AU02_int_qtl90', 'AU04_int_qtl90', 'AU05_int_qtl90', 'AU06_int_qtl90', 
                        'AU07_int_qtl90', 'AU09_int_qtl90', 'AU10_int_qtl90', 'AU12_int_qtl90', 'AU14_int_qtl90', 'AU15_int_qtl90', 
                        'AU17_int_qtl90', 'AU20_int_qtl90', 'AU23_int_qtl90', 'AU25_int_qtl90', 'AU26_int_qtl90', 'AU45_int_qtl90', 
                        'AU01_p_PerCent', 'AU02_p_PerCent', 'AU04_p_PerCent', 'AU05_p_PerCent', 'AU06_p_PerCent', 'AU07_p_PerCent', 'AU09_p_PerCent', 
                        'AU10_p_PerCent', 'AU12_p_PerCent', 'AU14_p_PerCent', 'AU15_p_PerCent', 'AU17_p_PerCent', 'AU20_p_PerCent', 'AU23_p_PerCent',
                        'AU25_p_PerCent', 'AU26_p_PerCent', 'AU45_p_PerCent']
    #print('featureNames: \n',featureNames)
    for feat in corrFeaturesList2:
        featIndices.append(featureNames.index(feat))
        
    testFeatureNames = [featureNames[i] for i in featIndices]
    testFeatureNames = [sub.replace('_Mean', '') for sub in testFeatureNames] 
    testFeatureNames = [sub.replace('_PerCent', '') for sub in testFeatureNames]
    testFeatureNames = [sub.replace('_meanVel', '') for sub in testFeatureNames]
    #print('testFeatureNames: \n',testFeatureNames)
    #print('featIndices: \n',featIndices)
    testFeatures = [features[i] for i in featIndices]
    #print('testFeatures: \n',testFeatures)
      
    #Create Activity Matrix
    #Compute the outer product to get how much activity in each pair of features
    activityMat = pd.DataFrame(np.outer(testFeatures,testFeatures),columns=testFeatureNames,index=testFeatureNames)
    #print('activityMat: \n',activityMat)
    
    #Spearman Correlation Matrix
    corDF = gazeDF.drop(['face_id','success','confidence','frame'],axis=1).corr(method='spearman')
    #print('corDF: \n',corDF)
    #print('corDF.columns: \n',corDF.columns)
    nanLessCorDF = corDF.fillna(0)
    
    #Partial Correlation Matrix
    partCorrDF = gazeDF.drop(['face_id','success','confidence','frame'],axis=1).pcorr()
    #print('partCorrDF: \n',partCorrDF)
    #print('partCorrDF.columns: \n',partCorrDF.columns)
    nanLessPartCorDF = partCorrDF.fillna(0)
    
    #Now add the Spearman correlation features to the current lists
    corrFeatureDF = nanLessCorDF.where(np.triu(np.ones(nanLessCorDF.shape),k=1).astype(bool))
    corrFeatureDF = corrFeatureDF.stack().reset_index()
    corrFeatureDF.columns = ['Row','Column','Value']
    #print('corrFeatureDF: \n',corrFeatureDF)
    corrFeatures = corrFeatureDF.Value
    corrFeatureNames = corrFeatureDF['Row'] + '_' + corrFeatureDF['Column']
    #print('corrFeatureNames: ',corrFeatureNames)
    #print('corrFeatures: \n',corrFeatures)

    
    #Plot Correlation Matrix
    plt.figure(figsize=(27,22))
    sns.heatmap(nanLessCorDF, annot=False, cmap='coolwarm', center=0.0, vmin=-1, vmax=1)
    corFileName = facePrintDir+Speaker+'_qtl_'+study+'_'+Session+'_AU_SpearCorr.png' 
    plt.savefig(corFileName, bbox_inches='tight')  
    #plt.show()
    plt.close()
    
    #Plot Activiy Matrix
    actFileName = facePrintDir+Speaker+'_qtl_'+study+'_'+Session+'_AU_Act.png' 
    corrplot(nanLessCorDF,activityMat,actFileName)
    
    return [corrFeatures,corrFeatureNames]    

def genActionUnitFeatures(openFaceDF):
    #anger_list = ['AU04_p','AU05_p','AU07_p','AU23_p']
    #anger_int_list = ['AU04_int_Mean','AU05_int_Mean','AU07_int_Mean','AU23_int_Mean']
    #anger_p_list = ['AU04_p_PerCent','AU05_p_PerCent','AU07_p_PerCent','AU23_p_PerCent']
    featureNames = list()
    features = list()
    totalMeanPresence = 0
    totalMeanIntensity = 0
    #Get rid of rows where the face wasn't detected successfully
    openFaceDF = openFaceDF.loc[openFaceDF['success']==1,:]
    rows,cols = openFaceDF.shape

    #Presence Metrics              
    presenceFeatureNames  = [f for f in newGazeCols if "_p" in f]
    for feature in presenceFeatureNames:
        newFeatureName =  feature+'_PerCent'
        featureNames.extend([newFeatureName])

        percentFeature = np.sum(openFaceDF.loc[:,feature])/rows
        totalMeanPresence = totalMeanPresence + percentFeature
        features.extend([percentFeature])

    #Intensity Metrics          
    intensityFeatureNames  = [f for f in newGazeCols if "_int" in f]
    for feature in intensityFeatureNames:
    
        newFeatureName =  feature+'_Mean'
        featureNames.extend([newFeatureName]) 
        meanFeature = np.mean(openFaceDF.loc[:,feature])
        totalMeanIntensity = totalMeanIntensity + meanFeature
        features.extend([meanFeature])
        
        newFeatureName =  feature+'_Max'
        featureNames.extend([newFeatureName])
        maxFeature = np.max(openFaceDF.loc[:,feature])
        features.extend([maxFeature])    
        
        newFeatureName =  feature+'_STD'
        featureNames.extend([newFeatureName])
        stdFeature = np.std(openFaceDF.loc[:,feature])
        features.extend([stdFeature]) 
        
        newFeatureName =  feature+'_Var'
        featureNames.extend([newFeatureName])
        varFeature = np.var(openFaceDF.loc[:,feature])
        features.extend([varFeature]) 
        
        newFeatureName =  feature+'_Mode'
        featureNames.extend([newFeatureName])
        modeFeature, counts = sp.stats.mode(openFaceDF.loc[:,feature])
        features.extend([modeFeature[0]]) 
        
        newFeatureName =  feature+'_Kur'
        featureNames.extend([newFeatureName])
        kurFeature = sp.stats.kurtosis(openFaceDF.loc[:,feature])
        features.extend([kurFeature]) 
        
        newFeatureName =  feature+'_Skew'
        featureNames.extend([newFeatureName])
        skewFeature = sp.stats.skew(openFaceDF.loc[:,feature])
        features.extend([skewFeature]) 
                
        newFeatureName =  feature+'_qtl05'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .05)
        features.extend([qtlFeature]) 
                        
        newFeatureName =  feature+'_qtl10'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .10)
        features.extend([qtlFeature]) 
                        
        newFeatureName =  feature+'_qtl25'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .25)
        features.extend([qtlFeature]) 
        
        newFeatureName =  feature+'_qtl50'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .50)
        features.extend([qtlFeature]) 
        
        newFeatureName =  feature+'_qtl75'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .75)
        features.extend([qtlFeature]) 
        
        newFeatureName =  feature+'_qtl90'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .90)
        features.extend([qtlFeature]) 
        
        newFeatureName =  feature+'_qtl95'
        featureNames.extend([newFeatureName])
        qtlFeature = np.quantile(openFaceDF.loc[:,feature], .95)
        features.extend([qtlFeature]) 
    
    #Global metrics        
    featureNames.extend(['AUs_int_Mean'])
    meanFeature = totalMeanIntensity/len(intensityFeatureNames)
    features.extend([meanFeature])  
    featureNames.extend(['AUs_p_Mean'])
    meanFeature = totalMeanPresence/len(presenceFeatureNames)
    features.extend([meanFeature]) 
    
    return [features,featureNames]  
    
def heatmap(x, y, actFileName, **kwargs):

    plt.figure(figsize=(22, 22))
    
    if 'color' in kwargs:
        color = kwargs['color']
    else:
        color = [1]*len(x)

    if 'palette' in kwargs:
        palette = kwargs['palette']
        n_colors = len(palette)
    else:
        n_colors = 32 # Use 32 colors for the diverging color palette
        palette = sns.color_palette("Blues", n_colors) 

    if 'color_range' in kwargs:
        color_min, color_max = kwargs['color_range']
    else:
        color_min, color_max = min(color), max(color) # Range of values that will be mapped to the palette, i.e. min and max possible correlation

    def value_to_color(val):
        if color_min == color_max:
            return palette[-1]
        else:
            val_position = float((val - color_min)) / (color_max - color_min) # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            ind = int(val_position * (n_colors - 1)) # target index in the color palette
            return palette[ind]

    if 'size' in kwargs:
        size = kwargs['size']
    else:
        size = [1]*len(x)

    if 'size_range' in kwargs:
        size_min, size_max = kwargs['size_range'][0], kwargs['size_range'][1]
        #print('size min: ',size_min)
        #print('size max: ',size_max)
    else:
        size_min, size_max = min(size), max(size)

    size_scale = kwargs.get('size_scale', 750)

    def value_to_size(val):
        if size_min == size_max:
            return 1 * size_scale
        else:
            val_position = (val - size_min) * 0.99 / (size_max - size_min) + 0.01 # position of value in the input range, relative to the length of the input range
            val_position = min(max(val_position, 0), 1) # bound the position betwen 0 and 1
            return val_position * size_scale
            
    if 'x_order' in kwargs: 
        x_names = [t for t in kwargs['x_order']]
    else:
        x_names = [t for t in sorted(set([v for v in x]))]
    x_to_num = {p[1]:p[0] for p in enumerate(x_names)}

    if 'y_order' in kwargs: 
        y_names = [t for t in kwargs['y_order']]
    else:
        y_names = [t for t in sorted(set([v for v in y]))]
    y_to_num = {p[1]:p[0] for p in enumerate(y_names)}

    plot_grid = plt.GridSpec(1, 15, hspace=0.2, wspace=0.1) 
    ax = plt.subplot(plot_grid[:,:-1]) # Use the left 14/15ths of the grid for the main plot
    ax.set_aspect('equal')

    #Square marker
    marker = kwargs.get('marker', 's')

    kwargs_pass_on = {k:v for k,v in kwargs.items() if k not in [
         'color', 'palette', 'color_range', 'size', 'size_range', 'size_scale', 'marker', 'x_order', 'y_order'
    ]}

    ax.scatter(
        x=[x_to_num[v] for v in x],
        y=[y_to_num[v] for v in y],
        marker=marker,
        s=[value_to_size(v) for v in size], 
        c=[value_to_color(v) for v in color],
        **kwargs_pass_on
    )
    ax.set_xticks([v for k,v in x_to_num.items()])
    ax.set_xticklabels([k for k in x_to_num], rotation=45, horizontalalignment='right')
    ax.set_yticks([v for k,v in y_to_num.items()])
    ax.set_yticklabels([k for k in y_to_num])

    ax.grid(False, 'major')
    ax.grid(True, 'minor', linewidth=0.1)
    ax.set_xticks([t + 0.5 for t in ax.get_xticks()], minor=True)
    ax.set_yticks([t + 0.5 for t in ax.get_yticks()], minor=True)

    ax.set_xlim([-0.5, max([v for v in x_to_num.values()]) + 0.5])
    ax.set_ylim([-0.5, max([v for v in y_to_num.values()]) + 0.5])
    ax.set_facecolor('#F1F1F1')
    
    plt.setp(ax.get_xticklabels(), fontsize=8)
    plt.setp(ax.get_yticklabels(), fontsize=8)

    # Add color legend on the right side of the plot
    if color_min < color_max:
        ax = plt.subplot(plot_grid[:,-1]) # Use the rightmost column of the plot

        col_x = [0]*len(palette) # Fixed x coordinate for the bars
        bar_y=np.linspace(color_min, color_max, n_colors) # y coordinates for each of the n_colors bars

        bar_height = bar_y[1] - bar_y[0]
        ax.barh(
            y=bar_y,
            width=[5]*len(palette), # Make bars 5 unit wide
            left=col_x, # Make bars start at 0
            height=bar_height,
            color=palette,
            linewidth=0
        )
        ax.set_xlim(1, 2) # Bars are going from 0 to 5, so lets crop the plot somewhere in the middle
        ax.grid(False) # Hide grid
        ax.set_facecolor('white') # Make background white
        ax.set_xticks([]) # Remove horizontal ticks
        ax.set_yticks(np.linspace(min(bar_y), max(bar_y), 3)) # Show vertical ticks for min, middle and max
        ax.yaxis.tick_right() # Show vertical ticks on the right 
          
    plt.savefig(actFileName, bbox_inches='tight')     
    plt.close()

def corrplot(data, sizeMat, actFileName, size_scale=500, marker='s'):
    corr = pd.melt(data.reset_index(), id_vars='index')
    corr.columns = ['x', 'y', 'value']
    #print('corr: \n',corr)
    activity = pd.melt(sizeMat.reset_index(), id_vars='index')
    activity.columns = ['x', 'y', 'value']
    #print('activity: \n',activity)
    heatmap(
        corr['x'], corr['y'],actFileName,
        color=corr['value'], color_range=[-1, 1],
        palette=sns.color_palette("coolwarm",32),
        size=activity['value'], size_range=[0,1.0],
        marker=marker,
        x_order=data.columns,
        y_order=data.columns[::-1],
        size_scale=size_scale
    )  
   


In [ ]:
# For each person in the dyad find their first order features


#Lists for ALL participants
faceFeaturesLists = list()
cohortPercentSuccessfulFramesList = list()
cohortAvgConfidenceSuccessfulFrameList  = list()
cohortSubjectSessionList  = list()

for Speaker in Speakers:
    speakerDir = InputDataDir + Speaker +'/'

    # Find all the Openface file for this Speaker.
    print('speakerDir: ',speakerDir)
    landmarkFiles = glob.glob(speakerDir+'**/TotalLandmarks.csv', recursive=True)
    print('landmarkFiles[0]: ',landmarkFiles)
    print('type(landmarkFiles[0]): ',type(landmarkFiles[0]))

    #Loop through all files under the directory
    firstTrip = False
    for faceFileName in sorted(landmarkFiles):
        print(faceFileName)

        #Read the full openface csv file
        faceDF = pd.read_csv(faceFileName) 
        faceFeaturesNameLists = list()

        #Get rid of leading blanks
        faceCols = list(faceDF)
        faceCols = [c.replace(' ', '') for c in faceCols]
        faceDF.columns = faceCols

        #Save the timestamps
        timestamps = faceDF['timestamp']
        print('type(timestamps): ',type(timestamps))

        #Subset to gaze/face columns
        gazeDF = faceDF[gazeCols]
        gazeDF.columns = newGazeCols

        #Summarize input and results
        frames,cols = gazeDF.shape
        print('OpenFace Video frames: ',frames)
        successfulFrameDF = gazeDF.loc[gazeDF.loc[:,"success"]==1,:]
        successfulFrameDF = successfulFrameDF.loc[successfulFrameDF.loc[:,"confidence"]>.88,:]
        numSuccessfulFrames, cols = successfulFrameDF.shape
        percentSuccessfulFrames = str(int((numSuccessfulFrames/frames)*100.0))
        avgConfidenceSuccessfulFrames = str(int(successfulFrameDF["confidence"].mean()*100.0))
        print('OpenFace successful frames: ',numSuccessfulFrames)
        print('Percent OpenFace successful frames: ',percentSuccessfulFrames)
        print('Average Confidence for Successful Frames: ',avgConfidenceSuccessfulFrames)

        #Lists for THIS participant
        faceFeaturesList = [Session, Diag, Participant, Speaker,percentSuccessfulFrames,avgConfidenceSuccessfulFrames]
        faceFeatureNamesList = ['Session', 'Diag', 'Participant', 'Speaker','percentSuccessfulFrames', 'avgConfidenceSuccessfulFrames']

        #                Gaze Metrics
        intGazeRX = (gazeDF['gaze_Rx'])
        intGazeRY = (gazeDF['gaze_Ry'])

        [features,featureNames] = genRobustMotionFeatures(intGazeRX ,gazeDF.loc[:,"success"],'gaze_Rx')
        faceFeaturesList.extend(features)
        faceFeatureNamesList.extend(featureNames)
        [features,featureNames] = genRobustMotionFeatures(intGazeRY ,gazeDF.loc[:,"success"],'gaze_Ry')
        faceFeaturesList.extend(features)    
        faceFeatureNamesList.extend(featureNames)

        #               Pose Metrics
        intPoseRX = (gazeDF['pose_Rx'])
        intPoseRY = (gazeDF['pose_Ry'])
        intPoseRZ = (gazeDF['pose_Rz'])
        [features,featureNames] = genRobustMotionFeatures(intPoseRX ,gazeDF.loc[:,"success"],'pose_Rx')
        faceFeaturesList.extend(features)
        faceFeatureNamesList.extend(featureNames)
        [features,featureNames] = genRobustMotionFeatures(intPoseRY ,gazeDF.loc[:,"success"],'pose_Ry')
        faceFeaturesList.extend(features)    
        faceFeatureNamesList.extend(featureNames)
        [features,featureNames] = genRobustMotionFeatures(intPoseRZ ,gazeDF.loc[:,"success"],'pose_Rz')
        faceFeaturesList.extend(features)    
        faceFeatureNamesList.extend(featureNames)

        #              Face Metrics
        [features,featureNames] = genActionUnitFeatures(gazeDF)
        faceFeaturesList.extend(features)
        faceFeatureNamesList.extend(featureNames)

        #               FacePrint (Correlation Metrics)
        #print('faceFeaturesList: \n',faceFeaturesList)
        #print('faceFeatureNamesList: \n',faceFeatureNamesList)
        facePrintDir = speakerDir + 'viz/'
        [features,featureNames] = genFacePrint(gazeDF, faceFeaturesList,faceFeatureNamesList)
        faceFeaturesList.extend(features)
        faceFeatureNamesList.extend(featureNames)

        #Append list for THIS participant to list for ALL participants
        faceFeaturesLists.append(faceFeaturesList)
        cohortPercentSuccessfulFramesList.append(percentSuccessfulFrames)
        cohortAvgConfidenceSuccessfulFrameList.append(avgConfidenceSuccessfulFrames)
        cohortSubjectSessionList.append(Session)

    print("SubjectSessionList: ",cohortSubjectSessionList)     
    print("cohortPercentSuccessfulFrameList: ",cohortPercentSuccessfulFramesList)   
    print("Mean cohortPercentSuccessfulFrames: ",np.mean(np.array(cohortPercentSuccessfulFramesList).astype(int)))
    print("Mean cohortAvgConfidenceSuccessfulFrameList: ",np.mean(np.array(cohortAvgConfidenceSuccessfulFrameList).astype(int)))


    #print("faceFeatureNamesList: ",faceFeatureNamesList)   
    
allStatsDF = pd.DataFrame(faceFeaturesLists,columns=faceFeatureNamesList)

outputFile = InputDataDir+'/summary_features/allRobustFaceStatsDF.csv'
print('Saving: '+outputFile)
allStatsDF.to_csv(outputFile, index=False)